In [18]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA
from matplotlib.pyplot import figure
import latex

In [36]:
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

plt.rcParams['text.usetex'] = True
plt.rcParams.update({
     "font.family": "serif",   # specify font family here
     "font.serif": ["Nimbus Roman"],  # specify font here
     "font.size":11})

#plt.style.use('seaborn-whitegrid')

In [47]:
def FigureSave(Time,Length,U_Gt,U_InitialSolve,time_pde,time_shock,Ndt,Initial,Cutoff,Alpha,TrainingSamples, S):
    #3 PDE figure
    fig = plt.figure(figsize=(8, 5), dpi=100)
                
    #plt.style.use('seaborn-whitegrid')
    
    rows = 1
    columns = 3
    
    fig.add_subplot(rows, columns, 1)
    
    U_PDE_FNO = np.append(U_InitialSolve[::Ndt,:],time_pde[:,:]).reshape(-1,S+7)
    X, T = np.meshgrid(Length,Time[::Ndt])            
    plt.pcolormesh(X,T,U_PDE_FNO,cmap=plt.cm.rainbow)
    #plt.imshow(U_PDE_FNO, interpolation='nearest', cmap='rainbow',origin='lower', aspect='auto')
    plt.colorbar(label=r'$u$')
    plt.grid(False)
    plt.title("FNO Solution")
    plt.xlabel(r'$x$')
    plt.ylabel(r'$t$')
    plt.tight_layout()
    #plt.savefig("Figures/" + f'PDEFNO_{Alpha}_{Ndt}_{TrainingSamples}.png')            
    
    
    
    fig.add_subplot(rows, columns, 2)
    
    #fig = figure(figsize=(6, 6), dpi=80)
    plt.pcolormesh(X,T,U_Gt[::Ndt,:],cmap=plt.cm.rainbow)
    #plt.imshow(U_Gt[::Ndt,:], interpolation='nearest', cmap='rainbow',origin='lower', aspect='auto')
    plt.colorbar(label=r'$u$')
    plt.title("Ground Truth Solution")
    plt.xlabel(r'$x$',fontsize = 'large')
    plt.ylabel(r'$t$',fontsize = 'large')
    plt.tight_layout()
    #plt.savefig("Figures/" + f'PDEGT_{Alpha}.jpg')
    
    fig.add_subplot(rows, columns, 3)
     
    #fig = figure(figsize=(6, 6), dpi=80)    
    error_PDE = U_PDE_FNO - U_Gt[::Ndt,:]
    plt.pcolormesh(X,T,error_PDE,cmap=plt.cm.rainbow)
    #plt.imshow(error_PDE, interpolation='nearest', cmap='rainbow',origin='lower', aspect='auto')
    plt.colorbar()
    plt.title('Pointwise Error')
    plt.xlabel(r'$x$',fontsize = 'large')
    plt.ylabel(r'$t$',fontsize = 'large')
    plt.tight_layout()
    plt.savefig("Figures/" + f'PDEerror_{Alpha}_{Ndt}_{TrainingSamples}.png')
    
    
    #1 Time Series
    fig = figure(figsize=(9, 5), dpi=80)
    U_Ex = np.append(U_InitialSolve[::Ndt,-1],time_shock[:,-1])
    plt.plot(Time[::Ndt], U_Gt[::Ndt,-1], 'grey',label='Exact')
    plt.plot(Time[::Ndt],U_Ex,'--',label = 'Predicted')
    plt.legend()
    #plt.legend([plot1,plot2],["Exact", "Predicted"])
    plt.title("Comparison",fontsize = 'large')
    plt.xlabel(r'$t$',fontsize = 'large')
    plt.ylabel(r'$u_s$',fontsize = 'large')
    plt.xlim([Initial,Cutoff])
    plt.grid(True)
    plt.ylim([0.85,1.3])
    
    
    plt.savefig("Figures/" + f'timeseries_{Alpha}_{Initial}_{Cutoff}_{Ndt}_{TrainingSamples}_{Initial}.png')
                
    #2 Error
    #pointwise error
    DT = Ndt*(Time[1]-Time[0])
    TotalTime = Time[::Ndt].shape[-1]
    error = (U_Ex - U_Gt[::Ndt,-1])

    InfError = LA.norm(error, np.inf)
    L2Norm = LA.norm(error, 2)*np.sqrt(DT/TotalTime)
                
    fig2 = figure(figsize=(9, 5), dpi=80)
    
    plt.plot(Time[::Ndt], error)
    plt.title("PointwiseError" + f'A_{Alpha}_{Ndt}')
    plt.xlabel('Time')
    plt.ylabel('Error')
    plt.xlim([Initial,Cutoff])
    
    plt.savefig("Figures/" + f'error_{Alpha}_{Initial}_{Cutoff}_{Ndt}_{TrainingSamples}_{Initial}.jpg')
                
    return InfError, L2Norm